<a href="https://colab.research.google.com/github/yogesh1q2w/PlanningGridWorld/blob/master/GridWorld_Tabular_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
from matplotlib import colors
from collections import defaultdict
import random

In [0]:
def printgrid(data):
  # create discrete colormap
  cmap = colors.ListedColormap(['red', 'blue','white','black'])
  bounds = [0,1,2,3,4]
  norm = colors.BoundaryNorm(bounds, cmap.N)

  fig, ax = plt.subplots()
  ax.imshow(data, cmap=cmap, norm=norm)

  # draw gridlines
  ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
  ax.set_xticks([])
  ax.set_yticks([])
  plt.show()

def matfromfile(file):
    with open(file,'r') as f:
        s = []
        t = []
        r = f.read()
        f.close()
        l = 0
        tt = 0
        for j in r:
            if(j == '\n'):
                s.append(t)
                t = []
                tt = 0
                l += 1
            else:
                if j == '2':
                    end = np.array([l,tt])
                t.append(int(j))
                tt += 1
        s.append(t)
        return np.array(s),end


In [0]:
class GridEnv:
    def __init__(self,file, det=True):
        self.gridfile = file
        self.determ = det
        self.grid, self.goal = matfromfile(self.gridfile) 
        self.nstates = np.size(self.grid)
        self.nactions = 4
        self.actions = ['l','r','u','d']
        self.nrows = np.shape(self.grid)[0]
        self.ncols = np.shape(self.grid)[1]
        self.End = False
        self.start = [np.random.randint(0,self.nrows),np.random.randint(0,self.ncols)]
        while(self._gridval(self.start) == 0 or self._isEnd(self.start)):
          self.start = [np.random.randint(0,self.nrows),np.random.randint(0,self.ncols)]
        self.curr_state = self.start
        
    def _gridval(self,state):
        return self.grid[state[0], state[1]]
        
    def _reward(self,state):
        
        if self.grid[state[0],state[1]] == 2:
            return +5
        elif self._gridval(state) == 0:   
            return -2
        else:
            return -1
        
    def _isEnd(self,state):
        if (self.grid[state[0], state[1]] == 2):
          self.End = True
        else:
          self.End = False
        return self.End
        
    def _nonDeterminsticTrans(self,action):
        if action == 'u':
            return np.random.choice(['u','l','r'], p = [0.8,0.1,0.1])
        
        if action == 'd':
            return np.random.choice(['d','l','r'], p = [0.8,0.1,0.1])
        
        if action == 'l':
            return np.random.choice(['l','u','d'], p = [0.8,0.1,0.1])
        
        if action == 'r':
            return np.random.choice(['r','u','d'], p = [0.8,0.1,0.1])
        
    def sense(self,k):
        s = self.curr_state
        up = np.flip(self.grid[max(0,s[0]-k):s[0],s[1]])
        down = self.grid[s[0]+1:min(self.nrows,s[0]+k+1),s[1]]
        left = np.flip(self.grid[s[0],max(0,s[1]-k):s[1]])
        right = self.grid[s[0],s[1]+1:min(self.ncols,s[1]+k+1)]
        
        if(np.size(up) > 0):
            zr = np.where(up == 0)
            if(np.size(zr) == 0):
                filtup = up
            else:
                filtup = up[0:np.amin(zr)]
        else:
            filtup = np.array([])
            
        if(np.size(down) > 0):
            zr = np.where(down == 0)
            if(np.size(zr) == 0):
                filtdown = down
            else:
                filtdown = down[0:np.amin(zr)]
        else:
            filtdown = np.array([])
            
        if(np.size(left) > 0):
            zr = np.where(left == 0)
            if(np.size(zr) == 0):
                filtleft = left
            else:
                filtleft = left[0:np.amin(zr)]
        else:
            filtleft = np.array([])
            
        if(np.size(right) > 0):
            zr = np.where(right == 0)
            if(np.size(zr) == 0):
                filtright = right
            else:
                filtright = right[0:np.amin(zr)]
        else:
            filtright = np.array([])
            
            
        return np.array([filtup,filtdown,filtleft,filtright])
        
    def transition(self, action):
        if self.determ:
            if action == 'u':
                nextState = self.curr_state + np.array([-1,0])
            elif action == 'd':
                nextState = self.curr_state + np.array([1,0])
            elif action == 'l':
                nextState = self.curr_state + np.array([0,-1])
            elif action == 'r':
                nextState = self.curr_state + np.array([0,1])
                
        else:
            action = self._nonDeterminsticTrans(action)
            self.determ = True
            nextState,_,_ = self.transition(action)
            self.determ = False
            
#         print(nextState)
        if (nextState[0] >= 0) and (nextState[0] < self.nrows):
            if (nextState[1] >= 0) and (nextState[1] < self.ncols):
              if (self._gridval(nextState) != 0):
                self.curr_state = nextState
                return nextState,self._reward(nextState),self._isEnd(nextState)
              else:
                return self.curr_state,self._reward(nextState),self._isEnd(self.curr_state)
        
        return self.curr_state, self._reward(self.curr_state), self._isEnd(self.curr_state)
             
    def reset(self):
        self.start = np.array([np.random.randint(0,self.nrows),np.random.randint(0,self.ncols)])
        while(self._gridval(self.start) == 0 or self._isEnd(self.start)):
          # print(self.start)
          self.start = np.array([np.random.randint(0,self.nrows),np.random.randint(0,self.ncols)])
        self.curr_state = self.start
        self.End = False
            

In [0]:
a = '(14,543)'
a = a[1:-1]
a.strip().split(',')

['14', '543']

In [0]:
def string(s):
  return '(' + str(s[0]) + ',' + str(s[1]) + ')'

def array(s):
  s = s[1:-1]
  l = s.strip().split(',')
  return np.array([int(l[0]), int(l[1])])

class SimpleAgent:
    
    def __init__(self, sense, goal):
        self.senseCap = sense
        self.actions = [0,1,2,3]
        self.actd = {0:'u',1:'d',2:'l',3:'r'}
        self.actrev = {'u':0,'d':1,'l':2,'r':3}
        self.model = np.ones((nrow,ncol))*3
        self.comps = DSU(nrow,ncol)
        self.goal = goal
        # print(goal)
        self.model[goal[0],goal[1]] = 2
        self.planbit = False                                                    #store if you are planning in current state
        
        self.lr = 0.2
        self.epsilon = 0.1
        self.gamma = 0.9
        self.planeps = 0.9
        self.Q_vals = defaultdict(lambda:[0.0,0.0,0.0,0.0])
        self.with_bonus = False                                                 #change this parameter if(not) using bonuses--> Bonus = max(all_rewards_recieved until now)+2.0
        self.inc_bonus = 2.0
        self.pseudo_rwd = self.inc_bonus
    
    def learn(self, s, a, r, next_s):
        a = self.actrev[a]
        current_q = self.Q_vals[s][a]
        if(self.comps.find(next_s) == self.comps.find(string(self.goal)) and self.planbit and self.with_bonus):
          r = max(r + self.inc_bonus, self.pseudo_rwd)
          self.planbit = False
        new_q = r + self.gamma * max(self.Q_vals[next_s])
        self.Q_vals[s][a] += self.lr * (new_q - current_q)

    def explore(self,state,senseRes):
      if np.random.rand() < self.epsilon:
          safeExps = []
          for i in range(0,4):
            if len(senseRes[i] > 0):
              safeExps.append(i)
                  
          action = np.random.choice(safeExps)
          acList = [self.actd[action]]
      else:
          state_action = self.Q_vals[state]
          action = self.arg_max(state_action)
          acList = [self.actd[action]]
      
      return acList

    def plan(self, state, senseRes):
      strgoal = string(self.goal)
      if self.comps.sameComp(state, strgoal):
        acList = self.BFS(array(state),self.goal)
      else:
        acList = self.borderBFS(array(state))
      
      self.planbit = True
      return acList

    def pick_action(self, state, senseRes):
      
        if np.random.rand() < self.planeps and (not self.isBorder(array(state)) or self.comps.sameComp(state,string(self.goal))):
          act1 = self.plan(state,senseRes)
          return act1

        else:
          act1 = self.explore(state,senseRes)
          return act1
          

    def BFS(self, source, goal):
      q = []
      q.append(source)
      pred = {}
      vis = defaultdict(lambda:0)
      vis[string(source)] = 1
      flagval = True
      while (not len(q) == 0 and flagval):
        u = q.pop(0)
        for i in [[-1,0],[1,0],[0,-1],[0,1]]:
          if not flagval:
            break
          coord = u+np.array(i)
          if(coord[0] >= 0 and coord[0] < nrow and coord[1] >=0 and coord[1] < ncol):
            if((self.model[coord[0],coord[1]] == 1 or self.model[coord[0],coord[1]] == 2)  and vis[string(coord)] == 0):
              q.append(coord)
              pred[string(coord)] = u
              vis[string(coord)] = 1

              if np.array_equal(coord,goal):
                flagval = False
                break
      
      acList = []
      v = goal 

      actionmap = {'(1,0)':'d','(-1,0)':'u','(0,1)':'r','(0,-1)':'l'}

      while(not np.array_equal(v,source)):
        acList = [actionmap[string(v-pred[string(v)])]] + acList
        v = pred[string(v)]

      return acList
      
    def isBorder(self,state):
      for i in [[-1,0],[1,0],[0,-1],[0,1]]:
          coord = state+np.array(i)
          if(coord[0] >= 0 and coord[0] < nrow and coord[1] >=0 and coord[1] < ncol):
            if(self.model[coord[0],coord[1]] == 3):
              return True
      return False

    def borderBFS(self, source):
      border_states = []
      q = []
      q.append(source)
      pred = {}
      vis = defaultdict(lambda:0)
      vis[string(source)] = 1
      flagval = True
      if(self.isBorder(source)):
        border_states.append(source)
      while (not len(q) == 0 and flagval):
        u = q.pop(0)
        for i in [[-1,0],[1,0],[0,-1],[0,1]]:
          if not flagval:
            break
          coord = u+np.array(i)
          if(coord[0] >= 0 and coord[0] < nrow and coord[1] >=0 and coord[1] < ncol):
            if((self.model[coord[0],coord[1]] == 1 or self.model[coord[0],coord[1]] == 2) and vis[string(coord)] == 0):
              q.append(coord)
              if(self.isBorder(coord)):
                border_states.append(coord)
              pred[string(coord)] = u
              vis[string(coord)] = 1

              if np.array_equal(coord,self.goal):
                flagval = False
                break
      
      acList = []
      # print(border_states)
      v = border_states[np.random.randint(0,len(border_states))] 

      actionmap = {'(1,0)':'d','(-1,0)':'u','(0,1)':'r','(0,-1)':'l'}

      while(not np.array_equal(v,source)):
        acList = [actionmap[string(v-pred[string(v)])]] + acList
        v = pred[string(v)]

      return acList


    @staticmethod
    def arg_max(state_action):
        '''
            Select a random action from a list of actions with
            equal Q values from a state
        '''
        max_index_list = []
        max_value = state_action[0]
        for index, value in enumerate(state_action):
            if value > max_value:
                max_index_list.clear()
                max_value = value
                max_index_list.append(index)
            elif value == max_value:
                max_index_list.append(index)
        return random.choice(max_index_list)

In [0]:
class DSU:

  def __init__(self, nrow, ncol):
    self.parent = {}
    self.rank = {}
    for i in range(0,nrow):
      for j in range(0,ncol):
        self.parent[string([i,j])] = string([i,j])
        self.rank[string([i,j])] = 0
  
  def find(self, elt):
    if self.parent[elt] != elt:
      self.parent[elt] = self.find(self.parent[elt])

    return self.parent[elt]

  def union(self,eltx, elty):
    xroot = self.find(eltx)
    yroot = self.find(elty)
    
    if (self.rank[xroot] < self.rank[yroot]):
      self.parent[xroot] = yroot
    elif (self.rank[xroot] > self.rank[yroot]):
      self.parent[yroot] = xroot
    else:
      self.parent[yroot] = xroot
      self.rank[xroot]+=1

  def sameComp(self,x,y):
    return self.find(x) == self.find(y)

In [0]:
nrow = 50
ncol = 50
env = GridEnv('50map.txt',1)
agent = SimpleAgent(3,env.goal)
n_epis = 10000
max_len = 10000
learn_steps = 5

plot_rwds = []
for episode in range(0,n_epis,learn_steps):

  for step in range(0,learn_steps):
    env.reset()
    epi_len  = 0
    while True:
      state = env.curr_state
      agent.model[state[0], state[1]] = 1
      # print(episode,state)
      sensorRes = env.sense(agent.senseCap)
      for i,x in enumerate(sensorRes):
        if i == 0:
          agent.model[state[0]-x.shape[0]:state[0],state[1]] = np.flip(x)
          for t in range(0,len(x)):
            agent.comps.union(string(state),string([state[0]-t-1,state[1]]))
          if(len(x) != agent.senseCap):
            agent.model[state[0]-x.shape[0]-1,state[1]] = 0
        elif i == 1:
          agent.model[state[0]+1:state[0]+x.shape[0]+1,state[1]] = x
          for t in range(0,len(x)):
            agent.comps.union(string(state),string([state[0]+t+1,state[1]]))
          if(len(x) != agent.senseCap):
            agent.model[state[0]+x.shape[0]+1,state[1]] = 0
        elif i == 2:
          agent.model[state[0], state[1]-x.shape[0]:state[1]] = np.flip(x)
          for t in range(0,len(x)):
            agent.comps.union(string(state),string([state[0],state[1]-t-1]))
          if(len(x) != agent.senseCap):
            agent.model[state[0],state[1]-x.shape[0]-1] = 0
        else:
          agent.model[state[0], state[1]+1:state[1]+x.shape[0]+1] = x
          for t in range(0,len(x)):
            agent.comps.union(string(state),string([state[0],state[1]+t+1]))
          if(len(x) != agent.senseCap):
            if state[1]+x.shape[0]+1 < ncol:
              agent.model[state[0],state[1]+x.shape[0]+1] = 0

      action = agent.pick_action(string(state),sensorRes)
      # print(action)

      # done = False

      for j in action:
        next_state, rwd, done = env.transition(j)
        agent.learn(string(state),j,rwd,string(next_state))

        state = next_state
        epi_len += 1
          
        if done or epi_len > max_len:
          done = True
          break
      
      # print('Step# ',step,'Epi_len=',epi_len)
      agent.model[state[0],state[1]] = 2
      printgrid(agent.model)
      agent.model[state[0],state[1]] = 1

      if done or epi_len > max_len:
        break

        

  env.reset()
  env.curr_state = env.start = np.array([48,1])
  epi_len = 0
  epi_rwd = 0
  while True:
    state = env.curr_state
    # print(episode,state)
    sensorRes = env.sense(agent.senseCap)
    action = agent.pick_action(string(state),sensorRes)
    done = False
    for j in action:
      next_state, rwd, done = env.transition(j)
      epi_rwd += rwd

      agent.learn(string(state),action,rwd,string(next_state))

      state = next_state
      epi_len += 1
          
      if done:
          plot_rwds.append(epi_rwd)
          break

    if done:
      break

  # printgrid(agent.model)

  print('Episode# ', episode, ' , Episode stats: ', epi_rwd, '/' , epi_len)

In [0]:
agent.borderBFS(np.array([12,37]))

['u', 'u', 'u', 'u', 'u', 'u', 'r', 'r', 'r', 'r', 'u', 'u']

In [0]:
agent.curr_state = np.array([1,47])
env.sense(3)

array([array([], dtype=int64), array([1, 1, 1]), array([1, 1, 1]),
       array([1, 1, 2])], dtype=object)

In [0]:
printgrid(env.grid)

In [0]:
v = [np.array([1]), np.array([2])]
v[np.random.randint(0,len(v))]

In [0]:
def proc(rews,k = 5):
    prews = []
    for i in range(0,len(rews),k):
        prews.append(sum(rews[i:i+k])/k)
    return prews

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
np.save('/content/drive/My Drive/Planning Project/plots/GridWorld-Deterministic-Model_50map.npy',plot_rwds)

In [0]:
rs = proc(plot_rwds,1)
plt.plot(rs,'r-')
plt.xlabel('Time')
plt.ylabel('Average Reward')

NameError: ignored

In [0]:
env.curr_state = np.array([1,1])
env.sense(3)

In [0]:
import numpy as np
plot_rwds = np.load('/content/drive/My Drive/Planning Project/plots/GridWorld-Deterministic-Model_50map.npy')
len(d)

2000